In [1]:
%reload_ext autoreload
%store -r volume old

In [2]:
%autoreload 2

import sys
sys.path.append(r'C:\Users\kmm171\ImageAnalysis\caImageAnalysis')
from natsort import natsorted
from scipy.stats import linregress
import numpy as np
import pandas as pd
import warnings
import seaborn as sns

#local imports
import main.main_eva as main_eva
import main.main_improv2 as improv
from utilities.pathutils import pathcrawler
from utilities.create_ColorDict import hex_to_RGB as hex_to_RGB
from utilities.arrutils import findmaxvalue
from caImageAnalysis.constants import monocular_dict

In [30]:
basefolder = r'C:\Users\kmm171\Desktop\Data\improv\training_data\20240125_7dpf'
fish1 = r'C:\Users\kmm171\Desktop\Data\improv\training_data\20240125_7dpf\139_eval3h2bgcamp6s_7dpf_fed_mono_R_20240125'
fish2 = r'C:\Users\kmm171\Desktop\Data\improv\training_data\20240125_7dpf\140b_eval3h2bgcamp6s_7dpf_fed_mono_R_20240125'
fish3 = r'C:\Users\kmm171\Desktop\Data\improv\training_data\20240125_7dpf\141_eval3h2bgcamp6s_7dpf_fed_mono_R_20240125'
fish4 = r'C:\Users\kmm171\Desktop\Data\improv\training_data\20240125_7dpf\142_eval3h2bgcamp6s_7dpf_fed_mono_R_20240125'
fish5 = r'C:\Users\kmm171\Desktop\Data\improv\training_data\20240125_7dpf\143_eval3h2bgcamp6s_7dpf_unfed_mono_R_20240125'
fish6 = r'C:\Users\kmm171\Desktop\Data\improv\training_data\20240125_7dpf\144_eval3h2bgcamp6s_7dpf_unfed_mono_R_20240125'

fishpaths = {

    "fish" + "139_a" : pathcrawler(fish1, set(), [], "img_stack_0"),
    "fish" + "139_b" : pathcrawler(fish1, set(), [], "img_stack_1"),
    "fish" + "140b_a" : pathcrawler(fish2, set(), [], "img_stack_0"),
    "fish" + "140b_b" : pathcrawler(fish2, set(), [], "img_stack_1"),
    "fish" + "141_a" : pathcrawler(fish3, set(), [], "img_stack_0"),
    "fish" + "141_b" : pathcrawler(fish3, set(), [], "img_stack_1"),
    "fish" + "142_a" : pathcrawler(fish4, set(), [], "img_stack_0"),
    "fish" + "142_b" : pathcrawler(fish4, set(), [], "img_stack_1"),
    "fish" + "143_a" : pathcrawler(fish5, set(), [], "img_stack_0"),
    "fish" + "143_b" : pathcrawler(fish5, set(), [], "img_stack_1"),
    "fish" + "144_a" : pathcrawler(fish6, set(), [], "img_stack_0"),
    "fish" + "144_b" : pathcrawler(fish6, set(), [], "img_stack_1"),

}

In [31]:
fishpaths

{'fish139_a': ['C:\\Users\\kmm171\\Desktop\\Data\\improv\\training_data\\20240125_7dpf\\139_eval3h2bgcamp6s_7dpf_fed_mono_R_20240125\\img_stack_0'],
 'fish139_b': ['C:\\Users\\kmm171\\Desktop\\Data\\improv\\training_data\\20240125_7dpf\\139_eval3h2bgcamp6s_7dpf_fed_mono_R_20240125\\img_stack_1'],
 'fish140b_a': ['C:\\Users\\kmm171\\Desktop\\Data\\improv\\training_data\\20240125_7dpf\\140b_eval3h2bgcamp6s_7dpf_fed_mono_R_20240125\\img_stack_0'],
 'fish140b_b': ['C:\\Users\\kmm171\\Desktop\\Data\\improv\\training_data\\20240125_7dpf\\140b_eval3h2bgcamp6s_7dpf_fed_mono_R_20240125\\img_stack_1'],
 'fish141_a': ['C:\\Users\\kmm171\\Desktop\\Data\\improv\\training_data\\20240125_7dpf\\141_eval3h2bgcamp6s_7dpf_fed_mono_R_20240125\\img_stack_0'],
 'fish141_b': ['C:\\Users\\kmm171\\Desktop\\Data\\improv\\training_data\\20240125_7dpf\\141_eval3h2bgcamp6s_7dpf_fed_mono_R_20240125\\img_stack_1'],
 'fish142_a': ['C:\\Users\\kmm171\\Desktop\\Data\\improv\\training_data\\20240125_7dpf\\142_eval3h2bgc

In [4]:
fishSchool = {}

for k, path in fishpaths.items():
    if volume:
        stackpaths = pathcrawler(f'{path[0]}', set(), [], "img_stack")
        for i, p in enumerate(stackpaths):
            f = improv.Fish(p, stimkey="output", old_stims=old)
            fishSchool[f'{k}_{i}'] = f
    else:
        for i, p in enumerate(fishpaths[k]):
            f = improv.Fish(p, stimkey="output", old_stims=old)
            fishSchool[f'{k}'] = f

#no frametimes? revise each txt file in img_stacks for empty spaces

In [5]:
fishSchool

{'fish163': <main.main_improv2.Fish at 0x26591b07070>,
 'fish164': <main.main_improv2.Fish at 0x26591b07160>,
 'fish165': <main.main_improv2.Fish at 0x26591b070d0>,
 'fish166': <main.main_improv2.Fish at 0x26591b070a0>,
 'fish167': <main.main_improv2.Fish at 0x26591b87c70>}

In [6]:
for key, stack in fishSchool.items():
    stack.parsePaths()
    basepath = stack.dataPaths['image'].parents[0].joinpath('suite2p')

    stack.dataPaths["suite2p"] = {
        "iscell": basepath.joinpath(
            "plane0/iscell.npy"
        ),
        "stats": basepath.joinpath(
            "plane0/stat.npy"
        ),
        "ops": basepath.joinpath(
            "plane0/ops.npy"
        ),
        "f_cells": basepath.joinpath(
            "plane0/F.npy"
        ),
        "f_neuropil": basepath.joinpath(
            "plane0/Fneu.npy"
        ),
        "spikes": basepath.joinpath(
            "plane0/spks.npy"
        ),
        "data": basepath.joinpath(
            "plane0/data.bin"
        ),
    }

In [7]:
offset = 10

responses_fishSchool = {}
errors_fishSchool = {}
bool_df_fishSchool = {}
responders_bool_fishSchool = {}

ops_fishSchool = {}
iscell_fishSchool = {}
stats_fishSchool = {}
f_cells_fishSchool = {}


for key, stack in fishSchool.items():
    ops, iscell, stats, f_cells = stack.load_suite2p(stack.dataPaths['suite2p'])
    f_cells = stack.new_norm_fdff(f_cells)

    neuron_responses = {}
    neuron_stds = {}

    for stimulus in stack.stimulus_df_condensed.stim_name.unique():
        stimmy_df = stack.stimulus_df_condensed[
            stack.stimulus_df_condensed.stim_name == stimulus
            ]

        starts = stimmy_df['motion_frame'].values

        allArrs = []
        for start_val in starts:
            stimArr = f_cells[:, start_val + 2 : start_val + 2 + offset]
            bgArr = f_cells[:, start_val - offset : start_val - 1]
            diffArr = np.nanmean(stimArr, axis=1) - np.nanmean(bgArr, axis=1)
            allArrs.append(diffArr)
        meanVals = np.nanmean(allArrs, axis=0)
        stdVals = np.nanstd(allArrs, axis=0)

        neuron_responses[stimulus] = meanVals
        neuron_stds[stimulus] = stdVals

    responses = pd.DataFrame(neuron_responses)
    errors = pd.DataFrame(neuron_stds)
    bool_df = responses > errors #Change responses >= errors
    responders = bool_df[bool_df.sum(axis=1) > 0]

    f_cells = pd.DataFrame(f_cells)

    ###
    responses_fishSchool[key] = responses
    errors_fishSchool[key] = errors
    bool_df_fishSchool[key] = bool_df
    responders_bool_fishSchool[key] = responders

    ops_fishSchool[key] = ops
    iscell_fishSchool[key] = iscell
    stats_fishSchool[key] = stats
    f_cells_fishSchool[key] = f_cells #these values are already normalized
    ###

#responders are already normalize. Each neuron has a max value, however, this max value might come from an outlier or from after the end of the trial or from the beginning of the recording. I would like to normalize each neuron response to highest response value across the entire experiment.


In [8]:
responders_f_cells_fishSchool = {}
#store in responders_fishSchool only those f_cells that are "responders"

for key, stack in fishSchool.items():

    responders_df = []
    responders_idx = responders_bool_fishSchool[key].index.tolist()

    for i, z in enumerate(responders_idx):
        responsive_f_cells = f_cells_fishSchool[key].iloc[z].values
        responders_df.append(responsive_f_cells)

    responders_f_cells_fishSchool[key] = pd.DataFrame(responders_df)

In [9]:
colors_fishSchool = {}

for key, fish in fishSchool.items():

    stimlist =  natsorted([e for e in fish.stimulus_df_condensed.stim_name.unique()])

    colors = {}
    color_dict = {}

    for i, n in enumerate(stimlist): #range(1,max_stim_size+1):
        color_dict[str(n)] = monocular_dict[i] #sns.color_palette("bright", n_colors=len(stimlist))[i]
    for k, v in color_dict.items():
        old_colors = list(v)
        fishSchool_alpha = [0.50]
        colors[k] = old_colors + fishSchool_alpha

    colors_fishSchool[key] = colors

In [10]:
colors_fishSchool['fish163']

{'1.79': [1, 0.25, 0, 0.5],
 '3.58': [0, 0.25, 1, 0.5],
 '5.36': [0, 1, 0, 0.5],
 '7.13': [1, 0, 1, 0.5],
 '14.04': [0, 0.75, 1, 0.5],
 '20.56': [0.75, 1, 0, 0.5],
 '23.63': [0.25, 0, 1, 0.5],
 '26.57': [1, 0, 0.25, 0.5],
 '43.15': [0.25, 1, 0, 0.5],
 '48.37': [0, 0.75, 1, 0.5]}

In [11]:
cell_img_fishSchool = {}

for key, stack in fishSchool.items():

    cell_img = np.zeros((ops_fishSchool[key]["Ly"], ops_fishSchool[key]["Lx"], 4), 'float64') #start with a blank image

    for row in range(len(responders_bool_fishSchool[key])):
        cell = responders_bool_fishSchool[key].iloc[row]
        nrn_color = [0,0,0,0]

        for stim in colors_fishSchool[key].keys():
            if cell.loc[float(stim)]:
                nrn_color = [nrn_color[i] + colors_fishSchool[key][stim][i] for i in range(len(nrn_color))]
            else:
                pass

        nrn_color = np.clip(nrn_color, a_min=0, a_max=1)
        ypix = stats_fishSchool[key][cell.name]['ypix']
        xpix = stats_fishSchool[key][cell.name]['xpix']

        for n, c in enumerate(nrn_color):
            cell_img[ypix, xpix, n] = c

    cell_img_fishSchool[key] = cell_img

In [12]:
stacked_resp_fishSchool = {}

offsetBaseline = 5
afterstim_frames = offset + 10

for key, fish in fishSchool.items():

    warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

    mini_stim = pd.DataFrame()
    mini_stim[['stim_name','motion']] = fish.stimulus_df_condensed[['stim_name','motion_frame']]

    nrns_df = pd.DataFrame()
    data = responders_f_cells_fishSchool[key]

    for neuron in range(len(data)): #a single neuron #Change: only the first 30 responders to go further
        all_resps_dict = {}
        all_resps_list = []

        for stim, group in mini_stim.groupby('stim_name'):

            groupedbystim = []

            for stimstart in group.motion:

                chunksize = len(data.iloc[neuron, mini_stim['motion'][0]-offsetBaseline : mini_stim['motion'][0]+afterstim_frames]) # compares every response to the first response of the same neuron. check the length of the first response to guide the size of the other vectors

                chunk = np.array(data.iloc[neuron, stimstart-offsetBaseline : stimstart+afterstim_frames])

                if len(chunk) < chunksize:
                    diff = chunksize - len(chunk)
                    chunk_sm = np.array(data.iloc[neuron, stimstart-offsetBaseline : stimstart+afterstim_frames+diff])
                    groupedbystim.append(chunk_sm)

                elif len(chunk) > chunksize:
                    diff =  len(chunk) - chunksize
                    chunk_lg = np.array(data.iloc[neuron, stimstart-offsetBaseline : stimstart+afterstim_frames-diff])
                    groupedbystim.append(chunk_lg)

                else:
                    chunk = np.array(data.iloc[neuron, stimstart-offsetBaseline : stimstart+afterstim_frames])
                    groupedbystim.append(chunk)

            all_resps_list.append(np.array(groupedbystim, dtype='object'))

        all_resps_dict = dict(zip(mini_stim['stim_name'].unique().tolist(),all_resps_list))
        singleneuron_df = pd.DataFrame({f'{neuron}' : all_resps_dict}).T #this is a big row = neuron with all trials
        nrns_df = pd.concat([nrns_df, singleneuron_df])
        nrns_df.columns = mini_stim['stim_name'].unique().tolist()

    stacked_resp_fishSchool[key] = nrns_df

testdata = responders_f_cells_fishSchool['fish136b'].iloc[4]
testdata = testdata[:].tolist()

#specify path for export
path = r'C:\Users\kmm171\Desktop\Data\improv\testdatafortuningcurve.txt'

#export DataFrame to text file
with open(path, 'w') as f:
    for line in testdata:
        f.write("%s\n" % line)

In [19]:
responsive_neurons_fishSchool = {}
responsive_bool_fishSchool = {}
stacked_responsive_fishSchool = {}

stimstart = offsetBaseline

for key, fish in fishSchool.items():
    stims = fish.stimulus_df_condensed['stim_name'].unique().tolist()
    data = stacked_resp_fishSchool[key]

    bool_df = pd.DataFrame()
    neuronresp = pd.DataFrame()
    responsive_neurons = pd.DataFrame()

    for index, neuron in data.iterrows():

        bool_resp = []
        neuron_resp = []

        for idx, chunk in neuron.items():
            #Find whether the peak avg is significantly different from baseline
            chunk = pd.DataFrame(chunk, dtype='float32') #this chunk has all the repetitions/trials
            chunk_avg = chunk.mean(axis=0)
            peak_index = chunk_avg.loc[stimstart:offset*2.5].idxmax()

            initialresp = chunk_avg.loc[stimstart:peak_index+5]
            peak = chunk_avg.loc[peak_index-8:peak_index+5]
            peak_avg = peak.mean()
            baseline_avg = chunk_avg.loc[:stimstart].mean()
            baseline_std = chunk_avg.loc[:stimstart].std()

            peakvalues = chunk[peak_index]
            variance = peakvalues.var()
            slope, intercept, r_value, p_value, std_err = linregress(x= range(len(initialresp)), y= initialresp)

            if peak_avg > 1.4*baseline_std + baseline_avg: # and variance < 0.12:
                if slope != 0:
                    bool_resp.append(1)
                    neuron_resp.append(peak_avg)
                else:
                    bool_resp.append(0)
                    neuron_resp.append(0)
            else:
                bool_resp.append(0)
                neuron_resp.append(0)

        bool_DICT = dict(zip(stims,bool_resp))
        singlebool_df = pd.DataFrame({f'{index}' : bool_DICT}).T
        bool_df = pd.concat([bool_df, singlebool_df])
        bool_df.columns = stims
        bool_df = bool_df.loc[bool_df.sum(1) > 0]

        resp_DICT = dict(zip(stims,neuron_resp))
        resp_df = pd.DataFrame({f'{index}' : resp_DICT}).T
        neuronresp = pd.concat([neuronresp, resp_df])
        neuronresp.columns = stims
        neuronresp = neuronresp.loc[neuronresp.sum(1) > 0]


    responsive_bool_fishSchool[key] = bool_df
    responsive_neurons_fishSchool[key] = neuronresp


    print(f"{key} length: {len(responsive_bool_fishSchool[key].index.tolist())}")

fish163 length: 72
fish164 length: 28
fish165 length: 43
fish166 length: 24
fish167 length: 34


In [20]:
stacked_responsive_fishSchool = {}

for key in fishSchool.keys():

    df1 = responsive_bool_fishSchool[key]
    df2 = stacked_resp_fishSchool[key]
    df3 = df2[df2.index.isin(df1.index)]

    stacked_responsive_fishSchool[key] = df3

In [21]:

avgs_fishSchool = {}
normsavgs_fishSchool = {}

for key in fishSchool.keys():
    data = stacked_resp_fishSchool[key]
    fishavg = pd.DataFrame()

    for idx in responsive_bool_fishSchool[key].index.tolist():
        neuron = stacked_resp_fishSchool[key].loc[idx]

        neuronavg = {}
        for size, chunk in neuron.items():
            chunkavg = np.mean(chunk,axis=0)
            neuronavg[size] = chunkavg

        neuronavg_df = pd.DataFrame({f'{idx}' : neuronavg}).T
        fishavg = pd.concat([fishavg, neuronavg_df])

    avgs_fishSchool[key] = fishavg



    #normalize each neuron to its own peak value
    fishnormavg = pd.DataFrame()

    for idx, row in fishavg.iterrows():
        maxvalue = row.explode().max()
        minvalue = row.explode().min()

        normneuron = {}
        for size, chunk in row.items():
            #normchunk = np.divide(chunk, maxvalue)
            normchunk = (chunk - minvalue) / (maxvalue - minvalue)
            normneuron[size] = normchunk

        normneuron_df = pd.DataFrame({f'{idx}' : normneuron}).T
        fishnormavg = pd.concat([fishnormavg, normneuron_df])

    normsavgs_fishSchool[key] = fishnormavg


In [22]:
cell_img_fishSchool = {}
for key, stack in fishSchool.items():
    cell_img = np.zeros((ops_fishSchool[key]["Ly"], ops_fishSchool[key]["Lx"], 4), 'float64') #start with a blank image

    for row in range(len(responders_bool_fishSchool[key])):
        cell = responders_bool_fishSchool[key].iloc[row]

        nrn_color = [0,0,0,0]

        for stim in colors_fishSchool[key].keys():
            if cell.loc[float(stim)]:
                nrn_color = [nrn_color[i] + colors_fishSchool[key][stim][i] for i in range(len(nrn_color))]
            else:
                pass

        nrn_color = np.clip(nrn_color, a_min=0, a_max=1)
        ypix = stats_fishSchool[key][cell.name]['ypix']
        xpix = stats_fishSchool[key][cell.name]['xpix']

        for n, c in enumerate(nrn_color):
            cell_img[ypix, xpix, n] = c

    cell_img_fishSchool[key] = cell_img

In [23]:
responsive_cell_img_fishSchool = {}
neurons_xypos= {}

for key, stack in fishSchool.items():
    cell_img = np.zeros((ops_fishSchool[key]["Ly"], ops_fishSchool[key]["Lx"], 4), 'float64') #start with a blank image

    xpos_coordinates = []
    ypos_coordinates = []
    cells_id = []
    cells_color = []

    for row in range(len(responsive_bool_fishSchool[key])):
        cell = responsive_bool_fishSchool[key].iloc[row]

        nrn_color = [0,0,0,0]

        for stim in colors_fishSchool[key].keys():
            if cell.loc[float(stim)]:
                nrn_color = [nrn_color[i] + colors_fishSchool[key][stim][i] for i in range(len(nrn_color))]
            else:
                pass

        nrn_color = np.clip(nrn_color, a_min=0, a_max=1)
        ypix = stats_fishSchool[key][int(cell.name)]['ypix']
        xpix = stats_fishSchool[key][int(cell.name)]['xpix']

        for n, c in enumerate(nrn_color):
            cell_img[ypix, xpix, n] = c

        cell_coordinates = stats_fishSchool[key][int(cell.name)]['med']

        xpos_coordinates.append(cell_coordinates[1])
        ypos_coordinates.append(cell_coordinates[0])
        cells_id.append(int(cell.name))
        cells_color.append(nrn_color)

    xypos_dict = {}
    xypos_dict['cell_id'] = cells_id
    xypos_dict['color'] = cells_color
    xypos_dict['xpos'] = xpos_coordinates
    xypos_dict['ypos'] = ypos_coordinates

    neurons_xypos[key] = xypos_dict
    responsive_cell_img_fishSchool[key] = cell_img

In [24]:
%store fishpaths fishSchool responses_fishSchool errors_fishSchool bool_df_fishSchool responders_bool_fishSchool responders_f_cells_fishSchool ops_fishSchool iscell_fishSchool stats_fishSchool f_cells_fishSchool offset stacked_resp_fishSchool offsetBaseline afterstim_frames colors_fishSchool cell_img_fishSchool responsive_neurons_fishSchool responsive_bool_fishSchool responsive_cell_img_fishSchool neurons_xypos stacked_resp_fishSchool normsavgs_fishSchool avgs_fishSchool stacked_responsive_fishSchool

Stored 'fishpaths' (dict)
Stored 'fishSchool' (dict)
Stored 'responses_fishSchool' (dict)
Stored 'errors_fishSchool' (dict)
Stored 'bool_df_fishSchool' (dict)
Stored 'responders_bool_fishSchool' (dict)
Stored 'responders_f_cells_fishSchool' (dict)
Stored 'ops_fishSchool' (dict)
Stored 'iscell_fishSchool' (dict)
Stored 'stats_fishSchool' (dict)
Stored 'f_cells_fishSchool' (dict)
Stored 'offset' (int)
Stored 'stacked_resp_fishSchool' (dict)
Stored 'offsetBaseline' (int)
Stored 'afterstim_frames' (int)
Stored 'colors_fishSchool' (dict)
Stored 'cell_img_fishSchool' (dict)
Stored 'responsive_neurons_fishSchool' (dict)
Stored 'responsive_bool_fishSchool' (dict)
Stored 'responsive_cell_img_fishSchool' (dict)
Stored 'neurons_xypos' (dict)
Stored 'stacked_resp_fishSchool' (dict)
Stored 'normsavgs_fishSchool' (dict)
Stored 'avgs_fishSchool' (dict)
Stored 'stacked_responsive_fishSchool' (dict)
